In [14]:
import pandas as pd
from gurobipy import *

In [22]:
df = pd.read_csv("./avgThenInterpolateMatrixF.csv")

In [23]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000


In [21]:
df.iloc[0][1]

0.0

In [41]:
model = Model("tree")
m = len(df)
n = len(df.columns)

delta = {0: [1, 10, 11, 12, 2, 3, 4, 5, 6],
 1: [8, 9],
 2: [13],
 3: [14, 15],
 4: [7],
 5: [],
 6: [],
 7: [16, 17, 18],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [19],
 19: []}

f = [ [model.addVar(vtype=GRB.CONTINUOUS, name="f_%d_%d" % (p, i)) for i in range(n)] for p in range(m)]
g = [ [model.addVar(vtype=GRB.CONTINUOUS, name="g_%d_%d" % (p, i)) for i in range(n)] for p in range(m)]
f_hat = [ [df.iloc[p][i] for i in range(n) ] for p in range(m) ]

for p in range(m):
    for i in range(n):
        model.addConstr(0 <= f[p][i] <= 1)
        model.addConstr(0 <= g[p][i] <= 1)
        rhs = quicksum([f[p][j] for j in delta[i]])
        if i > 0:
            model.addConstr(f[p][i] >= rhs)
            model.addConstr(g[p][i] >= f[p][i] - f_hat[p][i])
            model.addConstr(g[p][i] >= f_hat[p][i] - f[p][i])
        else:
            model.addConstr(f[p][i] >= rhs)
            model.addConstr(f[p][i] >= 0.8)
model.setObjective(quicksum(f[p][0] + g[p][i] for p in range(m) for i in range(1,n)), GRB.MINIMIZE)

In [42]:
model.optimize()

Optimize a model with 182952 rows, 73920 columns and 288288 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 68087 rows and 169421 columns
Presolve time: 1.09s
Presolved: 5833 rows, 13531 columns, 20394 nonzeros


Solved with dual simplex
Solved in 2966 iterations and 1.19 seconds
Optimal objective  2.824747403e+04


In [43]:
with open("F.csv", "w") as ff:
    for p in range(m):
        ff.write(",".join(map(str, [f[p][i].x for i in range(n)])) + "\n")

In [33]:
f_hat[0][4]

0.0

In [34]:
f[0][4].x

1.0

In [36]:
f_hat[0]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [37]:
model.write("test.lp")